In [1]:
import pandas as pd
df = pd.read_csv('../weather.csv')

In [2]:
df_dummies = pd.get_dummies(df, drop_first=True)

In [3]:
from sklearn.utils import shuffle

df_shuffled = shuffle(df_dummies, random_state=42)
df_shuffled

,Temperature_c,Humidity,Wind_Speed_kmh,Wind_Bearing_degrees,Visibility_km,Pressure_millibars,Rain,Description_Normal,Description_Warm
6252,-2.727778,0.51,25.2287,2,11.4471,1036.09,0,0,0
4684,11.094444,0.86,3.7030,283,9.9820,1017.35,1,1,0
1731,1.122222,0.88,12.1877,167,4.9105,1020.45,1,0,0
4742,-2.850000,0.64,10.4489,11,9.9820,1028.93,0,0,0
4521,7.777778,0.96,6.4400,180,3.0590,999.00,1,1,0
...,...,...,...,...,...,...,...,...,...
5734,8.888889,0.77,9.6600,20,9.9820,1019.50,1,1,0
5191,7.194444,0.65,23.7153,320,10.2557,1022.81,1,1,0
5390,1.088889,0.92,12.5419,21,6.3434,1026.62,1,0,0
860,31.344444,0.34,9.6761,174,9.9820,1017.01,1,0,1


In [4]:
DV = 'Rain'
X = df_shuffled.drop(DV, axis=1)
y = df_shuffled[DV]

In [5]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train,y_test = train_test_split(X,y,test_size=0.33,random_state=42)

In [6]:
from sklearn.linear_model import LogisticRegression

model = LogisticRegression()

In [7]:
model.fit(X_train,y_train)

C:\Users\JDhyeok\anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:763: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


LogisticRegression()

In [8]:
intercept = model.intercept_
coefficients = model.coef_

In [9]:
coef_list = list(coefficients[0,:])
coef_list

[6.145812818518533,
 -0.3183081501896287,
 -0.06804404295685355,
 -0.0023933653926554496,
 0.05595740609576024,
 0.0011022354948677113,
 0.099429025752099,
 0.04879622072147824]

In [10]:
coef_df = pd.DataFrame({'Feature': list(X_train.columns), 'Coefficient': coef_list})
print(coef_df)

                Feature  Coefficient
0         Temperature_c     6.145813
1              Humidity    -0.318308
2        Wind_Speed_kmh    -0.068044
3  Wind_Bearing_degrees    -0.002393
4         Visibility_km     0.055957
5    Pressure_millibars     0.001102
6    Description_Normal     0.099429
7      Description_Warm     0.048796


# Exercise26

In [16]:
predicted_prob = model.predict_proba(X_test)[:,1]
predicted_class = model.predict(X_test)
print(predicted_prob[:8])
print(predicted_class[:8])

[1.00000000e+00 1.00000000e+00 7.08699192e-06 1.00000000e+00
 1.00000000e+00 1.00000000e+00 1.00000000e+00 1.00000000e+00]
[1 1 0 1 1 1 1 1]


In [17]:
from sklearn.metrics import confusion_matrix
import numpy as np

cm = pd.DataFrame(confusion_matrix(y_test, predicted_class))
cm['Total'] = np.sum(cm, axis=1)
cm = cm.append(np.sum(cm,axis=0), ignore_index=True)
cm.columns = ['Predicted No', 'Predicted Yes', 'Total']
cm = cm.set_index([['Actual No', 'Actual Yes', 'Total']])
print(cm)

            Predicted No  Predicted Yes  Total
Actual No            377              6    383
Actual Yes            10           2907   2917
Total                387           2913   3300


In [19]:
from sklearn.metrics import classification_report
print(classification_report(y_test, predicted_class))

              precision    recall  f1-score   support

           0       0.97      0.98      0.98       383
           1       1.00      1.00      1.00      2917

    accuracy                           1.00      3300
   macro avg       0.99      0.99      0.99      3300
weighted avg       1.00      1.00      1.00      3300



# Exercise27

In [21]:
grid = {'penalty':['l1','l2'], 'C':np.linspace(1,10,10), 'solver':['liblinear']}

In [22]:
from sklearn.model_selection import GridSearchCV

model = GridSearchCV(LogisticRegression(solver='liblinear'),grid,scoring='f1',cv=5)

In [23]:
model.fit(X_train,y_train)

C:\Users\JDhyeok\anaconda3\lib\site-packages\sklearn\svm\_base.py:985: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn("Liblinear failed to converge, increase "
C:\Users\JDhyeok\anaconda3\lib\site-packages\sklearn\svm\_base.py:985: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn("Liblinear failed to converge, increase "
C:\Users\JDhyeok\anaconda3\lib\site-packages\sklearn\svm\_base.py:985: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn("Liblinear failed to converge, increase "
C:\Users\JDhyeok\anaconda3\lib\site-packages\sklearn\svm\_base.py:985: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn("Liblinear failed to converge, increase "
C:\Users\JDhyeok\anaconda3\lib\site-packages\sklearn\svm\_base.py:985: ConvergenceWarning: Liblinear failed to converge, increase the number of 

C:\Users\JDhyeok\anaconda3\lib\site-packages\sklearn\svm\_base.py:985: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn("Liblinear failed to converge, increase "
C:\Users\JDhyeok\anaconda3\lib\site-packages\sklearn\svm\_base.py:985: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn("Liblinear failed to converge, increase "
C:\Users\JDhyeok\anaconda3\lib\site-packages\sklearn\svm\_base.py:985: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn("Liblinear failed to converge, increase "
C:\Users\JDhyeok\anaconda3\lib\site-packages\sklearn\svm\_base.py:985: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn("Liblinear failed to converge, increase "
C:\Users\JDhyeok\anaconda3\lib\site-packages\sklearn\svm\_base.py:985: ConvergenceWarning: Liblinear failed to converge, increase the number of 

GridSearchCV(cv=5, estimator=LogisticRegression(solver='liblinear'),
             param_grid={'C': array([ 1.,  2.,  3.,  4.,  5.,  6.,  7.,  8.,  9., 10.]),
                         'penalty': ['l1', 'l2'], 'solver': ['liblinear']},
             scoring='f1')

In [28]:
best_parameters = model.best_params_
print(best_parameters)

{'C': 2.0, 'penalty': 'l1', 'solver': 'liblinear'}


In [32]:
predicted_prob = model.predict_proba(X_test)
predicted = model.predict(X_test)
print(predicted_prob)
print(predicted)

[[0.00000000e+000 1.00000000e+000]
 [0.00000000e+000 1.00000000e+000]
 [1.00000000e+000 2.65674521e-016]
 ...
 [1.00000000e+000 1.69866688e-102]
 [1.00000000e+000 5.98776859e-037]
 [0.00000000e+000 1.00000000e+000]]
[1 1 0 ... 0 0 1]


In [34]:
# make confusion matrics

cm = pd.DataFrame(confusion_matrix(y_test, predicted))
cm

,0,1
0,379,4
1,4,2913


In [37]:
cm['Total'] = np.sum(cm,axis=1)
cm = cm.append(np.sum(cm,axis=0),ignore_index=True)
cm.columns = ['Predicted No', 'Predicted Yes', 'Total']
cm = cm.set_index([['Actual No', 'Actual Yes', 'Total']])
cm

,Predicted No,Predicted Yes,Total
Actual No,379,4,1149
Actual Yes,4,2913,8751
Total,383,2917,9900
